# Setup

In [ ]:
from PIL import Image, ImageColor
from matplotlib.pyplot import imshow
import numpy as np

In [ ]:
!pip install memory_profiler

In [ ]:
import memory_profiler
import time

def time_mem_decorator(func):                                                                                            
    def out(*args, **kwargs):                                                                                            
        m1 = memory_profiler.memory_usage()
        t1 = time.time()
        
        result = func(*args, **kwargs)
        
        t2 = time.time()
        m2 = memory_profiler.memory_usage()
        time_diff = t2 - t1
        mem_diff = m2[0] - m1[0]
        print(f"It took {time_diff} Secs and {mem_diff} Mb to execute this function.")
        return(result)
    return out  

# Cirkels tekenen

Voor dit voorbeeld kijken we naar het handmatig tekenen van cirkels, door te bepalen welke pixels een kleur moeten krijgen gegeven een radius, center en lijn dikte.

In [ ]:
@time_mem_decorator
def draw_circles_naive(pixelmap, centers, radius, color, line_width = 5):
    for c in centers:
        for i in range(720):
            for l in range(line_width):
                x = float(c[0]) + float(radius + l) * np.sin(float(i) * np.pi / 360.0)
                y = float(c[1]) + float(radius + l) * np.cos(float(i) * np.pi / 360.0)
                pixelmap[round(x),round(y)] = ImageColor.getrgb(color)
        

In [ ]:
im = Image.new('RGB', (1000,1000), "white")
pixelmap = im.load()
imshow(im)

In [ ]:
radius = 50
n_circles = 200
centers = [(np.random.randint(radius+5, 1000-(radius+5)) ,np.random.randint(radius+5, 1000-(radius+5))) 
            for i in range(n_circles)]

draw_circles_naive(pixelmap, centers, 50, 'red')
imshow(im)

De sinus en cosinus operaties zijn hetzelfde voor elke lijn stukje op een bepaald deel van de cirkel, dus deze kunnen uit de binnenste for loop voor een versnelling.

In [ ]:
@time_mem_decorator
def draw_circles_faster(pixelmap, centers, radius, color, line_width = 5):
    for c in centers:
        for i in range(720):
            sin = np.sin(float(i) * np.pi / 360.0)
            cos = np.cos(float(i) * np.pi / 360.0)
            for l in range(line_width):
                x = float(c[0]) + float(radius + l) * sin
                y = float(c[1]) + float(radius + l) * cos
                pixelmap[round(x),round(y)] = ImageColor.getrgb(color)



In [ ]:
im = Image.new('RGB', (1000,1000), "white")
pixelmap = im.load()
imshow(im)

In [ ]:
radius = 50
n_circles = 200
centers = [(np.random.randint(radius+5, 1000-(radius+5)) ,np.random.randint(radius+5, 1000-(radius+5))) 
            for i in range(n_circles)]

draw_circles_faster(pixelmap, centers, 50, 'red')
imshow(im)

Als je meerdere cirkels tekent kan het nog sneller. Dit deel van de formule is voor elke cirkel hetzelfde. Je kunt dus 1 keer alle sinus en cosinus waardes uitrekenen en dit bewaren in een array. Dit uitlezen is sneller dan het uitrekenen!

In [ ]:
@time_mem_decorator
def draw_circles_fastest(pixelmap, centers, radius, color, line_width = 5):
    sins = [np.sin(float(i) * np.pi / 360.0) for i in range(720)]
    coss = [np.cos(float(i) * np.pi / 360.0) for i in range(720)]
    for c in centers:
        for i in range(720):
            for l in range(line_width):
                x = float(c[0]) + float(radius + l) * sins[i]
                y = float(c[1]) + float(radius + l) * coss[i]
                pixelmap[round(x),round(y)] = ImageColor.getrgb(color)

In [ ]:
im = Image.new('RGB', (1000,1000), "white")
pixelmap = im.load()
imshow(im)

In [ ]:
radius = 50
n_circles = 200
centers = [(np.random.randint(radius+5, 1000-(radius+5)) ,np.random.randint(radius+5, 1000-(radius+5))) 
            for i in range(n_circles)]

draw_circles_fastest(pixelmap, centers, 50, 'red')
imshow(im)